In [1]:
!pip install beautifulsoup4

In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
url = "https://www.okiden.co.jp/denki2/dl/"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

In [3]:
link_tags = soup.find_all("a", href=True)
csv_links = [link["href"] for link in link_tags if re.search(r'\.csv$', link["href"])]    

In [4]:
base_url = "https://www.okiden.co.jp/denki2/dl/"
csv_dict = {}

In [12]:
yearlyMonthly_links = soup.find_all("a", href=re.compile(r'^\./\d{4}/\d{6}\.html$'))

In [15]:
print(f'Found {len(yearlyMonthly_links)} months in total')

Found 98 months in total


In [33]:
yearlyMonthly_links[0]['href'][2:].split('.')

['2024/202405', 'html']

In [39]:
database = {}

In [ ]:
# https://www.okiden.co.jp/denki2/juyo_10_20240516.csv

In [79]:
for month in yearlyMonthly_links:
    sublink = month['href'][2:]
    year = int(sublink.split('/')[0])
    database[year] = {}
    month = int(sublink.split('.')[0][-2:])
    database[year][month] = {}
    monthlink = base_url + sublink
    monthly_response = requests.get(monthlink)
    monthly_soup = BeautifulSoup(monthly_response.content, "html.parser")
    link_div = monthly_soup.find("div", class_="mt-50")
    day_links = link_div.find_all("a", href=True)
    for day_link in day_links:
        single_day = day_link['href'].split('/')[-1]
        day = single_day.split('.')[-2][-2:]
        database[year][month][day] = None
        full_link = '/'.join(monthlink.split('/')[:-3])+'/'+single_day
        database[year][month][day] = full_link
        # print('Year {0}, month {1}, day {2}, link: {3}'.format(year, month, day, full_link))

In [80]:
len(database)

9

In [81]:
### database is nested dict where we have all links.

In [82]:
### Now lets download them and put to appropriate foders

In [86]:
import os

In [87]:
def download_csv_files(database):
    base_folder = "CSV_Files"
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)

    for year, months in database.items():
        year_folder = os.path.join(base_folder, str(year))
        if not os.path.exists(year_folder):
            os.makedirs(year_folder)

        for month, days in months.items():
            month_folder = os.path.join(year_folder, str(month).zfill(2))
            if not os.path.exists(month_folder):
                os.makedirs(month_folder)

            for day, url in days.items():
                response = requests.get(url)
                if response.status_code == 200:
                    file_name = f"{year}{str(month).zfill(2)}{str(day).zfill(2)}.csv"
                    file_path = os.path.join(month_folder, file_name)
                    with open(file_path, 'wb') as f:
                        f.write(response.content)
                    print(f"Downloaded: {file_path}")
                else:
                    print(f"Failed to download: {url}")


In [88]:
download_csv_files(database)


Downloaded: CSV_Files/2024/04/20240430.csv
Downloaded: CSV_Files/2024/04/20240429.csv
Downloaded: CSV_Files/2024/04/20240428.csv
Downloaded: CSV_Files/2024/04/20240427.csv
Downloaded: CSV_Files/2024/04/20240426.csv
Downloaded: CSV_Files/2024/04/20240425.csv
Downloaded: CSV_Files/2024/04/20240424.csv
Downloaded: CSV_Files/2024/04/20240423.csv
Downloaded: CSV_Files/2024/04/20240422.csv
Downloaded: CSV_Files/2024/04/20240421.csv
Downloaded: CSV_Files/2024/04/20240420.csv
Downloaded: CSV_Files/2024/04/20240419.csv
Downloaded: CSV_Files/2024/04/20240418.csv
Downloaded: CSV_Files/2024/04/20240417.csv
Downloaded: CSV_Files/2024/04/20240416.csv
Downloaded: CSV_Files/2024/04/20240415.csv
Downloaded: CSV_Files/2024/04/20240414.csv
Downloaded: CSV_Files/2024/04/20240413.csv
Downloaded: CSV_Files/2024/04/20240412.csv
Downloaded: CSV_Files/2024/04/20240411.csv
Downloaded: CSV_Files/2024/04/20240410.csv
Downloaded: CSV_Files/2024/04/20240409.csv
Downloaded: CSV_Files/2024/04/20240408.csv
Downloaded:

In [89]:
print("FINISHED")

FINISHED
